In [8]:
import pandas as pd

def process_age_groups(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 연령대 그룹 매핑 딕셔너리 생성
    age_mapping = {
        '19_24': '20대',
        '25_29': '20대',
        '30_34': '30대',
        '35_39': '30대',
        '40_44': '40대',
        '45_49': '40대',
        '50_54': '50대',
        '55_59': '50대',
        '60_80': '60대 이상'
    }
    
    # 원하는 연령대만 필터링
    df = df[df['age'].isin(age_mapping.keys())]
    
    # 새로운 연령대 컬럼 생성
    df['age_group'] = df['age'].map(age_mapping)
    
    # 일자별, 새로운 연령대별로 score 합산
    result = df.groupby(['date', 'name', 'age_group'])['score'].sum().reset_index()
    
    # 피벗 테이블로 변환하여 보기 좋게 정리
    pivot_result = result.pivot(index=['date', 'name'], 
                              columns='age_group', 
                              values='score').reset_index()
    
    # 컬럼 순서 정리
    column_order = [ 'date', 'name', '20대', '30대', '40대', '50대', '60대 이상']
    pivot_result = pivot_result[column_order]
    
    return pivot_result



In [9]:
# 광어
result_df = process_age_groups('../../data/raw/nst_광어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_광어_trend_2025-01-17.csv', index=False)

In [10]:
# 농어 
result_df = process_age_groups('../../data/raw/nst_농어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_농어_trend_2025-01-17.csv', index=False)

In [11]:
# 대게 
result_df = process_age_groups('../../data/raw/nst_대게_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_대게_trend_2025-01-17.csv', index=False)

In [12]:
# 방어 
result_df = process_age_groups('../../data/raw/nst_방어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_방어_trend_2025-01-17.csv', index=False)

In [13]:
# 연어
result_df = process_age_groups('../../data/raw/nst_연어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_연어_trend_2025-01-17.csv', index=False)

In [14]:
# 우럭 
result_df = process_age_groups('../../data/raw/nst_우럭_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_우럭_trend_2025-01-17.csv', index=False)

In [15]:
# 참돔 
result_df = process_age_groups('../../data/raw/nst_참돔_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_참돔_trend_2025-01-17.csv', index=False)

In [4]:
import pandas as pd

def calculate_item_daily_average(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 날짜와 어종별로 그룹화하여 시장 평균 계산
    daily_avg = df.groupby(['priceDate', 'item'])['avgPrice'].mean().reset_index()
    
    # 결측치를 앞뒤 값으로 채우기
    daily_avg = daily_avg.sort_values(['item', 'priceDate'])
    daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
    daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='bfill')
    
    return daily_avg

# 파일 처리
file_path = '../../data/processed/item_price_lag_filled.csv'
result_df = calculate_item_daily_average(file_path)

# 결과 저장
output_file = 'daily_item_average_prices.csv'
result_df.to_csv(output_file, index=False)
print(f"처리된 데이터가 {output_file}에 저장되었습니다.")

# 결과 미리보기
print("\n처리된 데이터 미리보기:")
print(result_df.head())

처리된 데이터가 daily_item_average_prices.csv에 저장되었습니다.

처리된 데이터 미리보기:
     priceDate item  avgPrice
19  2015-02-25   광어   25000.0
23  2015-02-26   광어   25000.0
27  2015-02-27   광어   25000.0
31  2015-02-28   광어   25000.0
35  2015-03-01   광어   25000.0


C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:12: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:13: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  daily_avg['avgPrice'] = daily_avg